# Implementation and evaluation of a computational standardization pipeline for chemical compounds

> Based on ["Trust, But Verify: On the Importance of Chemical Structure Curation in Cheminformatics and QSAR Modeling Research" from 2010 (D. Fourches, ...)"](https://pubmed.ncbi.nlm.nih.gov/20572635/)

By Allen Dumler; reviewed by Jaime Rodríguez-Guerra, PhD.

### Introduction 

This notebook serves to showcase the functionality of the `opencadd.compounds.standardization` subpackage. 

We are following the recommended standardization steps of "Trust, But Verify"(Fourches et al., 2010), and using the dataset of the following paper: [Cheminformatics Analysis of Assertions Mined from Literature That Describe Drug-Induced Liver Injury in Different Species](https://pubs.acs.org/doi/10.1021/tx900326k).

In [2]:
from pathlib import Path

HERE = Path(_dh[-1])
REPO = HERE.parents[1]

print("Tutorial location:", HERE)
print("Repo location:    ", REPO)

Tutorial location: /home/allen/dev/opencadd/docs/tutorials
Repo location:     /home/allen/dev/opencadd


In [4]:
# Import pandas and numpy
import pandas as pd
import numpy as np

# import modules and Standardization API functions needed
from rdkit import Chem

# from rdkit.Chem.PandasTools import RemoveSaltsFromFrame
from opencadd.compounds.standardization import (
    convert_format,
    handle_fragments,
    disconnect_metals,
    detect_inorganics,
    remove_salts,
    normalize,
    handle_tautomers,
    validate_molecules,
    detect_mixtures,
    detect_metals,
)

### Initial dataset import and cleaning of empty entries
------------------------------------------------
The first step before the standardization steps are started is a import of the dataset as an Pandas Dataframe, only including the columns necessary. In this case we use the <b>IDs</b>, <b>Names</b> and <b>SMILEs</b> column.<br>
Then we search for all entries which actually don't have any strings saved under <b>SMILEs</b> and kick them from the dataset, since they are not holding any information.<br>
After the import we add a <b>Failed_at</b> column to track in which standardization step the entry failed. 
The intial `task_number` will be 0, which leads to an default <b>Failed_at</b>-value of 0 for all entries, where null stands for <i>not failed</i> . 

In [5]:
task_number = 0

# Importing the test-dataset
dataset = pd.read_csv(HERE / "data" / "standardization_test_data.csv")

# Filter for needed columns
dataset = dataset[["IDs", "Names", "SMILEs"]]

# Kick all empty entries
empty_smiles = dataset[(dataset["SMILEs"].isnull())]

# The empty_smiles dataframe could be used to check which entires are affected and review the dataset again.
dataset = dataset[(dataset["SMILEs"].notna())]

# Setting a initial score of 0 for all entries in the 'Failed_at'-column
dataset["Failed_at"] = dataset["SMILEs"].apply(
    lambda x, task_number=task_number: task_number
)

dataset = dataset.reset_index(drop=True)
# Show the current form of the main-dataframe
dataset.tail(10)

,IDs,Names,SMILEs,Failed_at
194,195,Cinoxacin,CCN1N=C(C(O)=O)C(=O)c2cc3OCOc3cc12,0
195,196,Ciprofibrate,CC(C)(Oc1ccc(cc1)C1CC1(Cl)Cl)C(O)=O,0
196,197,Ciprofloxacin,OC(=O)C1=CN(C2CC2)c2cc(N3CCNCC3)c(F)cc2C1=O,0
197,198,Cisapride,COC1CN(CCCOc2ccc(F)cc2)CCC1NC(=O)c1cc(Cl)c(N)c...,0
198,199,Citalopram,CN(C)CCCC1(OCc2cc(ccc12)C#N)c1ccc(F)cc1,0
199,200,Citric acid,OC(=O)CC(O)(CC(O)=O)C(O)=O,0
200,201,zirconium,CCO[Zr](OCC)(OCC)OCC,0
201,202,hemoglobin,CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...,0
202,203,test_salt,[Al].N.[Ba].[Bi].Br.[Ca].Cl.F.I.[K].[Li].[Mg]....,0
203,204,test_duplicate,[Al].N.[Ba].[Bi].Br.[Ca].Cl.F.I.[K].[Li].[Mg]....,0


### Step 1: Conversion of SMILEs to mol
------------------------------------------

__Convert the SMILES representation format of the compounds into Mol-files__

RDKit performs a sanitization of the molecule by default. In this sanitization step RDKit tries to kekulize the mols (generates alternate Lewis structures). This step might fail, when the structure is aromatic, but no Hydrogen position is provided. 

> TODO: !(This explanation might be a bit short and not fully correct, check this later again)!

If the conversion from SMILE to mol fails, then those SMILEs will get a **Failed_at** marker added. 

To avoid the sanitization of the molecule `convert_smiles_to_mol` can be called with the argument `sanitize=False`. Keep in mind that the generation of different Lewis structures serves to find different representation formats of the same molecule. 

References:

* https://chemistry.stackexchange.com/questions/116498/what-is-kekulization-in-rdkit
* https://rdkit-discuss.narkive.com/QwnqcKcM/another-can-t-kekulize-mol-observation
* https://www.rdkit.org/docs/Cookbook.html
* https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html


#### Task 1: Convert to Mol

In [6]:
# Setting up the task_number
task_number = 1

# A column called mol is beeing added to the dataframe to store the mol-files
dataset["mol"] = dataset.loc[:, ("SMILEs")].apply(convert_format.convert_smiles_to_mol)

# Add task_number to failed entries
dataset.loc[dataset["mol"].isnull(), ["Failed_at"]] = task_number

dataset.head(16)

RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 7 9
RDKit ERROR: 
RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 2 3 4 6 7 8 10 11 12
RDKit ERROR: 
RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 6 8 10
RDKit ERROR: 
RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11 12 13 14 15
RDKit ERROR: 
RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 57 58 60
RDKit ERROR: 
RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 14 15 16 17 18 19 20 21 23
RDKit ERROR: 
RDKit ERROR: [18:52:44] Can't kekulize mol.  Unkekulized atoms: 11 12 13 15 16 17 19 20 21
RDKit ERROR: 


,IDs,Names,SMILEs,Failed_at,mol
0,1,(R)-Roscovitine,CCC(CO)Nc1nc(NCc2ccccc2)c2ncn(C(C)C)c2n1.[Ca],0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b40fe8a0>
1,2,17-Methyltestosterone,CC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC12C,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e210>
2,3,1-alpha-Hydroxycholecalciferol,CC(C)CCCC(C)C1CCC2C(CCCC12C)=CC=C1CC(O)CC(O)C1=C,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e1c0>
3,4,"2,3-Dimercaptosuccinic acid",OC(=O)C(S)C(S)C(O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e120>
4,5,"2,4,6-Trinitrotoluene",Cc1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e170>
5,6,2-Deoxy-D-glucose,OCC1OC(O)CC(O)C1O.O1CCOCC1,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e080>
6,7,2'-fluoro-5-methylarabinosyluracil,CC1=CN(C2OC(CO)C(O)C2F)C(=O)NC1=O,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e0d0>
7,8,2-Methoxyestradiol,COc1cc2C3CCC4(C)C(O)CCC4C3CCc2cc1O,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e2b0>
8,9,4-aminobenzoic acid,Nc1ccc(cc1)C(O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e3a0>
9,10,4-Hydroxytamoxifen,CCC(c1ccccc1)=C(c1ccc(O)cc1)c1ccc(OCCN(C)C)cc1,0,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e3f0>


### Step 2: Removal of Inorganics and Mixtures
--------------------------------------------------

Since  most  cheminformatical  applications  are  not  capable  of  processing  inorganic structures,  there  is the  need  for  a  removal  of  those  entries,  prior  to  any  processing.<br>
This is divided into two steps:<br>
First removing all entries not containing any Carbon at all, which are therefore not organic.<br>
Secondly filtering out all compounds with inorganic substructures. <br>

Similar problems occur for mixtures. Since most applications can not calculate descriptors for mixtures, a filtering has to happen prior to any processing. <br>
Additionally, since "*inorganic compounds are known to have biological effects, like for example toxic effects*" (Fourches 2010), we can often not distinguish if the recored activity of a mixed compound is caused by it's organic or inorganic part. Therefore the entry is useless and can be discarded. 


Due to the fact, that the treatment is not as simple as it apprears the paper (Fourches, 2010) recommends to delete records containing mixtures.
Common and widely used practice is to retain molecules with the highest molecular weight or the largest number of atoms(Chapter 2.1. Fourches 2010), but the paper (Fourches, 2010) states this might not be the best solution, and further investigation in mixtures should only be done if there is a reason to belive that the biological activity is really caused by the largest molecule and not the mixture itself.

#### Task 2: Filter entries without Carbon

To determine if a entry is a organic molecule, it obivious first task is to check for the presence of carbon. Therefore the `detect_carbon` function can be used. It checks for the presence of carbon atoms. If the functions finds at least one Carbon atom, it returns a boolean value of **True**, if not a value of **False**. All entries that return **False** will get a the number of the current task (2) written into the *Failed_at* column.
All entries that already have failed in another step won't the considered in this step, since they are already disqualified for further analysis.

In [7]:
# Setting up the task_number
task_number = 2

# Check for Carbon
dataset["Carbon_present"] = dataset.apply(
    lambda row: detect_inorganics.detect_carbon(row.mol)
    if row.Failed_at == 0
    else None,
    axis=1,
)

# Add task_number to failed entries
dataset.loc[dataset["Carbon_present"] == False, ["Failed_at"]] = task_number

Below you can see all entries, that don't contain any Carbon and thereby it can be assumed that they are inorganic molecules.

In [8]:
dataset[dataset["Failed_at"] == 2]

,IDs,Names,SMILEs,Failed_at,mol,Carbon_present
202,203,test_salt,[Al].N.[Ba].[Bi].Br.[Ca].Cl.F.I.[K].[Li].[Mg]....,2,<rdkit.Chem.rdchem.Mol object at 0x7fb0b407f760>,False
203,204,test_duplicate,[Al].N.[Ba].[Bi].Br.[Ca].Cl.F.I.[K].[Li].[Mg]....,2,<rdkit.Chem.rdchem.Mol object at 0x7fb0b407f7b0>,False


#### Task 3: Filter entries with inorganic components

While we filtered out all molecules not containing any Carbon, now we further inspect the entries for elements which can not or only rarely occur in organic molecules or are contained by them. This might vary a bit depending on the defenition and scope. For this we can use the `detect_inorganic` function.
It is recommended to check what can be handled by software for later use of the dataset. A customization of the allowed elements in `detect_inorganic` is possible, and can easily be provided by a set of SMARTS, as described shortly further below. 
As the default set of accepted elements in a organic molecule Hydrogen, Carbon, Nitrogen, Oxygen, Fluorine, Phosphorus, Sulfur, Chlorine, Selenium, Bromine, Iodine (nonmetals and halogenes) were chosen. <br>
*While Astatine and Tennessine are also considered halogenes, they are not included due to their radioactivity and rarity.*
<br>


###### An example on how a custom set can be setup and how it can be used 
-----------------------------------------------------------------------------------------------------------
Defining a set:<br>
`elements = Chem.MolFromSmarts("[!#1&!#6&!#7&!#8&!#9&!#15&!#16&!#17&!#35&!#53]")`

Pass the set as a parameter, where the `detect_inorganic` function is getting called:<br>
`lambda row: detect_inorganics.detect_inorganic(row.mol, elements)`

In [9]:
# Setting up the task_number
task_number = 3


# Check for inorganic structures
dataset["Inorganics"] = dataset.apply(
    lambda row: detect_inorganics.detect_inorganic(row.mol)
    if row.Failed_at == 0
    else None,
    axis=1,
)

# Add task_number to failed entries
dataset.loc[dataset["Inorganics"] == True, ["Failed_at"]] = task_number

Below you can see all entries, that contain other than our allowed elements.(Hydrogen, Carbon, Nitrogen, Oxygen, Fluorine, Phosphorus, Sulfur, Chlorine, Selenium, Bromine, Iodine)

In [10]:
dataset[dataset["Failed_at"] == 3]

,IDs,Names,SMILEs,Failed_at,mol,Carbon_present,Inorganics
0,1,(R)-Roscovitine,CCC(CO)Nc1nc(NCc2ccccc2)c2ncn(C(C)C)c2n1.[Ca],3,<rdkit.Chem.rdchem.Mol object at 0x7fb0b40fe8a0>,True,True
114,115,Bortezomib,CC(C)CC(NC(=O)C(Cc1ccccc1)NC(=O)c1cnccn1)B(O)O,3,<rdkit.Chem.rdchem.Mol object at 0x7fb0b407ac10>,True,True
200,201,zirconium,CCO[Zr](OCC)(OCC)OCC,3,<rdkit.Chem.rdchem.Mol object at 0x7fb0b407f6c0>,True,True
201,202,hemoglobin,CC1=C(C2=CC3=NC(=CC4=C(C(=C([N-]4)C=C5C(=C(C(=...,3,<rdkit.Chem.rdchem.Mol object at 0x7fb0b407f710>,True,True


#### Task 4: Filter entries containing mixtures

Since mixtures appear in encoding formats, like SMILES strings, where various molecules can be stored in one entry, but many applications can not handle mixtures as descriptors, they also need to be filtered out.
Some mixtures can be used, when it can be clearly identified, that the recored activity originates from them and common fragments, without bioactivity are known, but prior to any action on the entries with mixtures, those have to be identified.
This will be archived with the function `detect_mixtures` as shown below.

In [11]:
# Setting up the task_number
task_number = 4

# Check for inorganic structures
dataset["mixture"] = dataset.apply(
    lambda row: detect_mixtures(row.mol) if row.Failed_at == 0 else None,
    axis=1,
)

# Add task_number to failed entries
dataset.loc[dataset["mixture"] == True, ["Failed_at"]] = task_number

Below you can see all entries, that are mixtures.

In [12]:
dataset[dataset["Failed_at"] == 4]

,IDs,Names,SMILEs,Failed_at,mol,Carbon_present,Inorganics,mixture
5,6,2-Deoxy-D-glucose,OCC1OC(O)CC(O)C1O.O1CCOCC1,4,<rdkit.Chem.rdchem.Mol object at 0x7fb0b410e080>,True,False,True


#### Task 5: Filter entries containing metals

In [18]:
# Setting up the task_number
task_number = 5

# Check for metals
dataset["metals"] = dataset.apply(
    lambda row: detect_metals(row.mol) if row.Failed_at == 0 else None,
    axis=1,
)

# Add task_number to failed entries
dataset.loc[dataset["mixture"] == True, ["Failed_at"]] = task_number

Below you can see all entries, containing metals

In [20]:
dataset[dataset["Failed_at"] == 4]

,IDs,Names,SMILEs,Failed_at,mol,Carbon_present,Inorganics,mixture,metals


### Step 3: Structural Conversion and Cleaning
--------------------------------------------------

Some drugs need to be transformed "into their salt form to enhance how the drug disscolves (...) and (to) increase it's effectiveness. (https://www.drugs.com/article/pharmaceutical-salts.html (03/12/21)) Therefore it is common for chemical compound databases to contain records of salts. If possible it is recommended to delete the records containing salts completely, since, similar to in-organic compounds, "most descriptor-generating software (can not process salts)" (Fourches 2010 Chapter 2.2 ).While not being desirable, it is still an acceptable procedure to convert compounds into their neutral forms. But cases like this should be tagged, filtered and afterwards manually curated or compared to the actual neutral form of that compound. 
In case that we want to continue working on the converted records, we should perform the following steps:
- check if records contain compounds with presence of metals --> difficult case, filter out (already done this - one step ahead)
- removing the salts from the record
- neutralize the record (normalization or basic standardization)
- neutralize the charges
- to be discussed: the adding/removing of hydrogens, both got pros and cons (pro addingH --> higher prediction performances / con addingH --> may introduce noise --> less reliable models)(removingH might introduce erros in calculating descriptors, due to it might not handle certain cases well)



In [ ]:
# Structural coversion
# Cleaning/removal of salts
# Functions remove_salts
# normalize_molecules
# handle_charges
# handle_hydrogens

#### Task 6: Removing salts 

In [ ]:
# Setting up the task_number
task_number = 6
# getting the valid entries from the step before
dataset = result1  # Load results 1 here, to see the functionality, 'cause all entries containing salts, have already been filtered by prior steps.

# Create Smiles for evaluation
dataset["smiles_before"] = dataset["mol"].apply(convert_format.convert_mol_to_smiles)

# Perform disconnect_metals on entries
dataset["mol"] = dataset["mol"].map(remove_salts)

# Create new SMILEs from the current state for evaluation of performed changes
dataset["Smiles 5"] = dataset["mol"].apply(convert_format.convert_mol_to_smiles)
dataset["no_removed_salt"] = dataset["smiles_before"] == dataset["Smiles 5"]

# Filter the changed entries
changed_at_step_6 = dataset[dataset["no_removed_salt"] == False]
changed_at_step_6["Changed_at"] = changed_at_step_6["Failed_at"].apply(
    lambda x, task_number=task_number: task_number
)
changed_at_step_6 = changed_at_step_6[["IDs", "Names", "SMILEs", "Changed_at", "mol"]]

dataset["Changed_at"] = dataset[dataset["no_removed_salt"] == False]["Failed_at"].apply(
    lambda x, task_number=task_number: task_number
)
dataset.head(20)

# dataset['removed_salts'] = RemoveSaltsFromFrame(dataset,molCol='mol')
# where_salt = dataset[dataset['removed_salts'].notna()]

In [ ]:
# Show the subset of all changed entries (salts were removed)
changed_at_step_6.head()

#### Task 7: Normalize molecules

In [ ]:
# TODO: Finish all steps here

# Setting up the task_number
task_number = 7
# getting the valid entries from the step before
dataset = result1
# dataset.head(100)
dataset["normalized"] = dataset["mol"].apply(normalize)
result7 = dataset
result7.head()

#### Task 8: Charges and Hydrogens TODO

In [ ]:
# TODO: Add the functionlaity here

### Normalization of Specific Chemotypes

More complex than just Normalization.

In [ ]:
# TODO: Finish all steps here

# Normalization of specific chemotypes
# normalize_molecules

In [ ]:
# TODO: Finish all steps here

# Treatment of tautomeric forms
# handle_tautomers

#### Task 9: Generate a canonicalized tautomer on SMILEs entries

In [ ]:
# TODO: Finish all steps here

# Setting up the task_number
task_number = 9

dataset = result7

# Find all duplicate occurences in SMILEs
dataset["canonicalized tautomer"] = dataset["SMILEs"].apply(
    handle_tautomers.canonicalize_tautomer
)
dataset.head()

In [ ]:
dataset.tail()

### Removal of duplicates

In [ ]:
# TODO: Fine tune the output

# Analysis/removal of duplicates

# Setting up the task_number
task_number = 10

dataset = result7

# Find all duplicate occurences in SMILEs
dataset["duplicate?"] = dataset.duplicated(subset=["SMILEs"])

# Filter the duplicates out
failed_step_10 = dataset[dataset["duplicate?"] == True]
failed_step_10["Failed_at"] = failed_step_10["Failed_at"].apply(
    lambda x, task_number=task_number: task_number
)

dataset = dataset[dataset["duplicate?"] == False]
failed_step_10.tail()

In [ ]:
# Manual inspection

# TODO: Create csv-exports for better readability of the subsets or jupyter notebook searchable tables

In [ ]:
# Contatination of results for the end
test = pd.concat([failed_step_1, failed_step_2])
test = test.sort_values(by=["IDs"])

In [ ]:
test